In [71]:
import pandas as pd
import minsearch
import os
from groq import Groq
from dotenv import load_dotenv


In [73]:
# Load environment variables from .env file
load_dotenv()
GROQ_API_KEY="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"

In [40]:
file_path = '../data/data.csv'
df = pd.read_csv(file_path)


In [49]:
#df=df.drop("id",axis=1)
df.head()

,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."


## Indexing it using Minsearch

In [10]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
!curl -O https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3832  100  3832    0     0   5492      0 --:--:-- --:--:-- --:--:--  5537


In [50]:
df.columns.str.lower()

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [56]:
documents =df.to_dict(orient='records')

In [57]:
documents

[{'exercise_name': 'Push-Ups',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Upper Body',
  'type': 'Push',
  'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
  'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'},
 {'exercise_name': 'Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Stand with feet shoulder-width apart. Lower your body as if sitting back into a chair, keeping your chest up. Return to standing.'},
 {'exercise_name': 'Plank',
  'type_of_activity': 'Strength/Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Core',
  'type': 'Hold',
  'muscle_groups_activated': 'Rectus Abdominis, Transverse Abdominis',
  'instructions': '

In [54]:
Index=minsearch.Index(
    text_fields=['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=[]
)

In [58]:
Index.fit(documents)

In [91]:
query="What should I eat if I have kidney infection"

In [77]:
Index.search(query,num_results=2)

[{'exercise_name': 'Kettlebell Goblet Squat',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Kettlebell',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Hold a kettlebell at chest level and squat down as if sitting in a chair. Return to standing.'},
 {'exercise_name': 'Goblet Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Kettlebell',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Hold a kettlebell at chest level and squat down as if sitting in a chair. Return to standing.'}]

## Implementing LLM

In [78]:

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

When you have a kidney infection, it's essential to follow a diet that helps manage the infection, alleviates symptoms, and prevents further complications. Here are some dietary recommendations to consider:

1. **Stay hydrated**: Drink plenty of water (at least 8-10 glasses a day) to help flush out bacteria and toxins from your kidneys. You can also consume electrolyte-rich beverages like coconut water, sports drinks, or Pedialyte to replenish lost electrolytes.
2. **Consume electrolyte-rich foods**: Include foods rich in electrolytes like potassium, sodium, and phosphorus in your diet. These foods can help maintain electrolyte balance:
	* Bananas (potassium)
	* Avocados (potassium)
	* Nuts and seeds (magnesium, potassium)
	* Leafy greens (magnesium, potassium)
	* Whole grains (magnesium)
	* Canned fish (sodium)
3. **Avoid certain foods**: Limit or avoid foods that can irritate your kidneys or worsen symptoms:
	* **High-sodium foods**: Excessive sodium can exacerbate inflammation and i

In [79]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [86]:
def search(query):
    boost = {}

    results = Index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=5
    )

    return results

In [83]:
documents[0]

{'exercise_name': 'Push-Ups',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Bodyweight',
 'body_part': 'Upper Body',
 'type': 'Push',
 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}

In [88]:
entry_template = """
exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part: {body_part}
type: {type}
muscle_groups_activated: {muscle_groups_activated}
instructions: {instructions}
""".strip()

prompt_template = """
You're a fitness insrtuctor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.
QUESTION: {question}

CONTEXT: 
{context}
""".strip()


def build_prompt(query, search_results):

    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc)+ "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [94]:
def llm(prompt):
    response = client.chat.completions.create(
        model='llama3-8b-8192',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


In [98]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [99]:
query="I want excercises that help my glute grow"

In [100]:
search_results = search(query)
prompt =build_prompt(query,search_results)
answer=rag(query)
print(answer)

Based on the provided context, I'd recommend the following exercises to help your glutes grow:

1. **Glute Kickback**: This exercise specifically targets the glutes and hamstrings, which is perfect for growing your glutes.
2. **Glute Bridge**: This exercise also targets the glutes and hamstrings, which can help to grow and strengthen your glutes.
3. **Single-Leg Glute Bridge**: This exercise is a variation of the Glute Bridge that targets the glutes and hamstrings, and can help to improve balance and stability.

I would not recommend the Push Press or Kettlebell Push Press exercises, as they target the upper body muscles (deltoids and triceps) rather than the glutes.
